<a href="https://colab.research.google.com/github/praffuln/agentic-ai/blob/master/memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('helloworld')

helloworld


# Lesson 2: Baseline Email Assistant

This lesson builds an email assistant that:
- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings

We'll start with a simple implementation - one that uses hard-coded rules to handle emails.

![Memory Course App](./img/memory_course_email.png)


### Setup Environments


In [ ]:
!pip install google-generativeai tiktoken chromadb langgraph langchain langchain_community langchainhub ipykernel langchain_groq sentence_transformers boto3 langchain_aws docx exceptions langchain_openai langchain-google-vertexai
!pip install -U langchain-google-vertexai
!pip install --upgrade python-docx


 ## Setup Keys For GOOGLE_API_KEY


In [3]:

 # Assigning value to variable
GEMINI_API_KEY=''
SERPER_API_KEY = ''



# setup environments


In [4]:
import os
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY




## setup LLM

In [5]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)
llm = genai.GenerativeModel("gemini-1.5-flash")
response = llm.generate_content("Explain how AI works")
print(response.text)




Artificial intelligence (AI) is a broad field encompassing many techniques, but at its core, it's about creating systems that can perform tasks that typically require human intelligence. These tasks include learning, reasoning, problem-solving, perception, and natural language understanding.  There's no single "how" it works, as different AI approaches utilize different methods.  However, we can break it down into key concepts:

**1. Data is King:**  AI systems learn from data.  The more relevant and high-quality data you feed an AI, the better it performs. This data can be anything from images and text to sensor readings and financial transactions.

**2. Algorithms: The Recipes for Learning:**  Algorithms are sets of rules and statistical techniques that AI systems use to analyze data and learn patterns.  These algorithms are the "brains" of the operation, dictating how the system processes information and makes decisions.  Different algorithms are suited to different tasks.

**3. Mac

## Setup a Profile, Prompt and Example Email

In [6]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [7]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently."
}

In [8]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

## Define the first part of the agent - triage.

In [9]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated
from langchain.chat_models import init_chat_model

In [ ]:
!pip install langchain_google_genai -U
from langchain_google_genai import GoogleGenerativeAI
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser # Import PydanticOutputParser


class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

# Wrap the Google Generative AI model with LangChain's wrapper
# Use `model` instead of `model_name` to specify the model
llm = genai.GenerativeModel("gemini-1.5-flash") # This line is replaced


# Define the prompt template
prompt_template = """Analyze the unread email and route it according to its content.
Email: {email}
Triage Rules:
- Ignore: "{ignore}"
- Notify: {notify}
- Respond: {respond}

Reasoning:
"""  # The model will complete this with its reasoning

# Create a PydanticOutputParser instance
output_parser = PydanticOutputParser(pydantic_object=Router) # Create an instance of PydanticOutputParser

# Create an LLMChain for structured output
# llm_router = LLMChain(
#     llm=llm,
#     prompt=PromptTemplate.from_template(prompt_template),
#     output_parser=output_parser  # Pass the PydanticOutputParser instance
# )


In [25]:
# Agent prompt baseline
agent_system_prompt = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

# Agent prompt semantic memory
agent_system_prompt_memory = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
4. manage_memory("email_assistant", user, "collection") - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
5. manage_memory("email_assistant", user, "user_profile") - Store any relevant information about the recipient, {name}, in the user profile for future reference the current user profile is shown below
6. search_memory("email_assistant", user, "collection") - Search memory for detail from previous emails
7. manage_memory("email_assistant", user, "instructions") - Update the instructions for agent tool usage based upon the user feedback
</ Tools >

< User profile >
{profile}
</ User profile >

< Instructions >
{instructions}
</ Instructions >
"""

# Triage prompt
triage_system_prompt = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few shot examples >
"""

triage_user_prompt = """
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}"""


In [26]:
system_prompt = triage_system_prompt.format(
    full_name=profile["full_name"],
    name=profile["name"],
    examples=None,
    user_profile_background=profile["user_profile_background"],
    triage_no=prompt_instructions["triage_rules"]["ignore"],
    triage_notify=prompt_instructions["triage_rules"]["notify"],
    triage_email=prompt_instructions["triage_rules"]["respond"],
)

In [27]:
user_prompt = triage_user_prompt.format(
    author=email["from"],
    to=email["to"],
    subject=email["subject"],
    email_thread=email["body"],
)

In [14]:
print(system_prompt)


< Role >
You are John Doe's executive assistant. You are a top-notch executive assistant who cares about John performing as well as possible.
</ Role >

< Background >
Senior software engineer leading a team of 5 developers. 
</ Background >

< Instructions >

John gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that John should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from John

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
Marketing newsletters, spam emails, mass company announcements

There are also other things that John should know about, but don't require an email response. For these, you should notify John (using the `notify` response). Examples of this include:
Team member out sick, build system notifications, pro

In [15]:
print(user_prompt)


Please determine how to handle the below email thread:

From: Alice Smith <alice.smith@company.com>
To: John Doe <john.doe@company.com>
Subject: Quick question about API documentation

Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice


In [33]:
llm = genai.GenerativeModel("gemini-1.5-flash")
prompt = system_prompt + "\n" + user_prompt # Concatenate system and user prompts
response = llm.generate_content(prompt) # Use generate_content to generate response
print(response.text) # Print the generated text



RESPOND

